In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [35]:
from sklearn.metrics import accuracy_score

In [6]:
data = pd.read_csv(r"C:\Users\HP\Downloads\diabetes.csv")

In [7]:
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [8]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
data.hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000000000B833710>,
      dtype=object)

In [10]:
y = data["Outcome"]
X = data.drop(["Outcome"],axis=1)

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [12]:
train_mean_pos = X_train[y_train==1].mean()
train_std_pos = X_train[y_train==1].std()
train_mean_neg = X_train[y_train==0].mean()
train_std_neg = X_train[y_train==0].std()

In [13]:
summary = {"train_mean_pos":train_mean_pos.tolist(),"train_std_pos":train_std_pos.tolist(),
           "train_mean_neg": train_mean_neg.tolist(), 
           "train_std_neg": train_std_neg.tolist()}

In [14]:
summary

{'train_mean_pos': [4.805714285714286,
  141.57714285714286,
  69.88,
  20.634285714285713,
  101.52571428571429,
  34.896571428571434,
  0.5313657142857143,
  37.25714285714286],
 'train_std_pos': [3.7900309992623353,
  31.432897746713344,
  23.19587755684583,
  17.87321155520765,
  136.21474835093554,
  6.808867588117155,
  0.32437997970208815,
  11.092979184696874],
 'train_mean_neg': [3.430939226519337,
  110.6353591160221,
  68.70441988950276,
  19.4585635359116,
  68.75966850828729,
  30.438674033149216,
  0.4402734806629832,
  31.370165745856355],
 'train_std_neg': [3.111400483034198,
  26.488060729846165,
  17.746707620851563,
  14.948654360273792,
  98.1358390793149,
  7.330310950072529,
  0.3149715984444194,
  11.685680062880774]}

In [22]:
import numpy as np
def cond_prob(x,mean,std):
    variance = std*std
    p = 1/(np.sqrt(2*np.pi*variance))*np.exp((-(x-mean)**2)/(2*variance))
    return p

In [18]:
def posterior_prob(row, summary):
    pos_prob = len(X_train[y_train==1])/(len(X_train))
    neg_prob = len(X_train[y_train==0])/(len(X_train))
    
    for i in range(0,len(row)):
        pos_prob = pos_prob*cond_prob(row[i],train_mean_pos[i],train_std_pos[i])
    for i in range(0,len(row)):
        neg_prob = neg_prob*cond_prob(row[i],train_mean_neg[i],train_std_neg[i])
    return [pos_prob,neg_prob]

In [31]:
predicted_raw = []
for row in X_test.values.tolist():
    predicted_raw.append(posterior_prob(row,summary))

In [32]:
predicted_raw

[[1.2643247829470507e-13, 1.6917919343546252e-12],
 [1.8186526483377514e-13, 1.36546405658521e-13],
 [2.3272270842409015e-15, 7.229141082905065e-17],
 [3.6359415100624446e-13, 8.258255509063782e-13],
 [9.520752199975288e-14, 4.008547338090255e-14],
 [1.319035306378098e-20, 8.698032233810916e-19],
 [6.06390956675677e-14, 2.211212412729573e-12],
 [3.7330934411738546e-14, 1.3688977678220218e-14],
 [2.1076987515160663e-14, 1.4706321532028631e-12],
 [1.3982977835926985e-13, 7.793426623304716e-13],
 [1.0329685943806582e-13, 1.0670347736600489e-12],
 [6.764852344560664e-14, 1.4849104831857285e-12],
 [1.8406008772030164e-13, 3.696874501516992e-12],
 [1.4986533898047282e-14, 1.1188076286763924e-13],
 [6.919310963291199e-14, 1.9978955321829156e-14],
 [2.373583841927446e-13, 1.2022814197613075e-12],
 [4.313925869665761e-15, 1.184071730034644e-13],
 [1.3889120938793537e-13, 1.1230346744788962e-13],
 [2.9458786203217845e-14, 1.2500316108653639e-12],
 [4.7427836711651477e-14, 6.362339151732403e-14],

In [33]:
predictions = []
for row in predicted_raw:
    if(row[0]>row[1]):
        predictions.append(1)
    else:
        predictions.append(0)

In [34]:
confusion_matrix(y_test.tolist(),predictions)

array([[121,  17],
       [ 40,  53]], dtype=int64)

In [36]:
accuracy_score(y_test.tolist(),predictions)

0.7532467532467533